In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from pandas import DataFrame
from datetime import datetime
import os
import glob

In [ ]:
out_filepath = f'./evaluation_results/results{datetime.now().strftime("%Y-%m-%d_%H%M%S")}.csv'
with open(out_filepath, 'a') as out_file:
    out_file.write('name,auc,accuracy,precision,recall')

folders = [
    './network_output/stylegan2/ffhq/',
    './network_output/stylegan2/car/',
    './network_output/stylegan2/cat/',
    './network_output/stylegan2/church/',
    './network_output/stylegan2/horse/',

    './network_output/stylegan3/ffhq/',
    './network_output/stylegan3/afhq_v2/',
    './network_output/stylegan3/metfaces/',

    './network_output/gated_convolution_inpainting/',

    './network_output/glide/',    

    './network_output/taming-transformers/'
]

In [ ]:
all_data_frames = {
    'stylegan2': [], 
    'stylegan3': [],
}

def evaluate(folder):
    plt.clf()

    files = sum([glob.glob(os.path.join(folder, '*.csv'))], list())
    if len(files) == 0: return
    
    for file in files:
        data = pd.read_csv(file)
        name = file.split('/')[-1][22:-4]

        if 'stylegan3' in name:            
            all_data_frames['stylegan3'].append(data)
        elif 'stylegan2' in name:
            all_data_frames['stylegan2'].append(data)

        auc = metrics.roc_auc_score(data["fake"], data["logit"])

        fpr, tpr, _ = metrics.roc_curve(data["fake"], data["logit"])

        plt.plot(fpr,tpr, label=f'{name} AUC:{round(auc, 6)}')
        
        pred = data["logit"].map(lambda x: 1 if x >= 0 else 0)

        accuracy = metrics.accuracy_score(data["fake"], pred)
        precision = metrics.precision_score(data["fake"], pred)
        recall = metrics.recall_score(data["fake"], pred)

        with open(out_filepath, 'a') as out_file:
            out_file.write(f'{name},{auc},{accuracy},{precision},{recall}\n')
            print(name, "ROC-AUC", auc, "Accuracy", accuracy, "Precision", precision, "Recall", recall)        
    
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc="lower right")

    
    base_name = folder.split('/')[-3]+'-' if 'stylegan' in folder else ''
    out_name =  base_name + folder.split('/')[-2]
    plt.savefig(f"evaluation_results/{out_name}.png")
    plt.show()

In [ ]:
for x in folders:
    evaluate(x)

In [ ]:
for key, value in all_data_frames.items():  
    all_data = pd.concat(value)

    auc = metrics.roc_auc_score(all_data["fake"], all_data["logit"])
    fpr, tpr, _ = metrics.roc_curve(all_data["fake"], all_data["logit"])

    pred = all_data["logit"].map(lambda x: 1 if x >= 0 else 0)
    
    accuracy = metrics.accuracy_score(all_data["fake"], pred)
    precision = metrics.precision_score(all_data["fake"], pred)
    recall = metrics.recall_score(all_data["fake"], pred)

    with open(out_filepath, 'a') as out_file:
        out_file.write(f'{key},{auc},{accuracy},{precision},{recall}\n')
        print(key, "ROC-AUC", auc, "Accuracy", accuracy, "Precision", precision, "Recall", recall)   
    
    plt.plot(fpr,tpr, label=f'{key}-all-categories AUC:{round(auc, 6)}')

    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc="lower right")
    plt.savefig(f"evaluation_results/{key}-all.png")
    plt.show()